In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from mpl_toolkits.axes_grid1 import ImageGrid
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16


In [2]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [3]:
CATEGORIES = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed', 'Common wheat', 'Fat Hen', 'Loose Silky-bent',
              'Maize', 'Scentless Mayweed', 'Shepherds Purse', 'Small-flowered Cranesbill', 'Sugar beet']
NUM_CATEGORIES = len(CATEGORIES)
NUM_CATEGORIES

12

In [4]:
print("Total images count:")
for category in CATEGORIES:
    print('{} {} images'.format(category, len(os.listdir(os.path.join("../NonsegmentedV2", category)))))

Total images count:
Black-grass 309 images
Charlock 452 images
Cleavers 335 images
Common Chickweed 713 images
Common wheat 253 images
Fat Hen 538 images
Loose Silky-bent 762 images
Maize 257 images
Scentless Mayweed 607 images
Shepherds Purse 274 images
Small-flowered Cranesbill 576 images
Sugar beet 463 images


%pip install split_folders

In [5]:
import splitfolders
splitfolders.ratio('../NonsegmentedV2/', output="../output", seed=1337, ratio=(.8, 0.1,0.1))

Copying files: 5539 files [01:50, 50.16 files/s]


In [6]:
train_folder = "../output/train"
NUM_TRAIN_EXAMPLES = 0
for _, _, files in os.walk(train_folder):
    NUM_TRAIN_EXAMPLES += len([fn for fn in files if fn.endswith(".png")])
test_folder = "../output/test"
NUM_TEST_EXAMPLES = 0
for _, _, files in os.walk(test_folder):
    NUM_TEST_EXAMPLES += len([fn for fn in files if fn.endswith(".png")])
val_folder = "../output/val"
NUM_VAL_EXAMPLES = 0
for _, _, files in os.walk(val_folder):
    NUM_VAL_EXAMPLES += len([fn for fn in files if fn.endswith(".png")])

In [7]:
print("Trainging images count:")
for category in CATEGORIES:
    print('{} {} images'.format(category, len(os.listdir(os.path.join(train_folder, category)))))

Trainging images count:
Black-grass 247 images
Charlock 361 images
Cleavers 268 images
Common Chickweed 570 images
Common wheat 202 images
Fat Hen 430 images
Loose Silky-bent 609 images
Maize 205 images
Scentless Mayweed 485 images
Shepherds Purse 219 images
Small-flowered Cranesbill 460 images
Sugar beet 370 images


In [8]:
image_size = (150, 150, 3)

fig = plt.figure(1, figsize=(NUM_CATEGORIES, NUM_CATEGORIES))
grid = ImageGrid(fig, 111, nrows_ncols=(NUM_CATEGORIES, NUM_CATEGORIES), axes_pad=0.05)
i = 0
for category_id, category in enumerate(CATEGORIES):
    for filepath in train[train['category'] == category]['file'].values[:NUM_CATEGORIES]:
        ax = grid[i]
        img = read_img(filepath, (image_size[0], image_size[1]))
        ax.imshow(img / 255.)
        ax.axis('off')
        if i % NUM_CATEGORIES == NUM_CATEGORIES - 1:
            ax.text(250, 112, filepath.split('/')[1], verticalalignment='center')
        i += 1
plt.show()

In [9]:
def train_preprocess():
    # Preprocessing
    TRAINING_DIR = "../output/train"
    train_datagen = ImageDataGenerator(rescale=1./255., 
                                    rotation_range=40, 
                                    width_shift_range=0.2, 
                                    height_shift_range=0.2, 
                                    shear_range=0.2, zoom_range=0.2, 
                                    horizontal_flip=True)
    # TRAIN GENERATOR.
    train_generator = train_datagen.flow_from_directory(
                        TRAINING_DIR,
                        target_size =(150, 150),
                        batch_size = 32,
                        class_mode = 'categorical')

    return train_generator

In [10]:
def test_preprocess(DIR):
    # Preprocessing
    # VALIDATION GENERATOR.
    validation_datagen = ImageDataGenerator(rescale=1./255.)
    validation_generator = validation_datagen.flow_from_directory(
                        DIR,
                        target_size =(150, 150),
                        batch_size = 10,
                        class_mode = 'categorical')
    return validation_generator

In [11]:
def solution_model():
    train_ds = train_preprocess()
    val_ds = test_preprocess("../output/val")
    test_ds = test_preprocess("../output/test")
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=image_size),
        MaxPooling2D(2, 2),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Conv2D(256, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Flatten(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(12, activation="softmax")
    ])
    model.compile(
        loss='categorical_crossentropy',
        optimizer=Adam(),
        metrics=['accuracy']
    )
    model.fit(
        train_ds,
        epochs=30,
        steps_per_epoch=NUM_TRAIN_EXAMPLES // 32,
        validation_data=val_ds,
        validation_steps=NUM_VAL_EXAMPLES//10
    )
    model.evaluate(test_ds, steps=NUM_TEST_EXAMPLES//10)
    return model 

In [12]:
if __name__ == "__main__":
    model = solution_model()
    model.save('../Model/Plant_Seedling.h5')


Found 4426 images belonging to 12 classes.
Found 548 images belonging to 12 classes.
Found 565 images belonging to 12 classes.
Epoch 1/30
138/138 [==============================] - 199s 1s/step - loss: 2.1133 - accuracy: 0.2685 - val_loss: 1.6347 - val_accuracy: 0.4648
Epoch 2/30
138/138 [==============================] - 114s 825ms/step - loss: 1.6018 - accuracy: 0.4320 - val_loss: 1.5051 - val_accuracy: 0.4333
Epoch 3/30
138/138 [==============================] - 116s 839ms/step - loss: 1.4102 - accuracy: 0.5000 - val_loss: 1.1770 - val_accuracy: 0.6185
Epoch 4/30
138/138 [==============================] - 116s 842ms/step - loss: 1.2573 - accuracy: 0.5555 - val_loss: 1.0694 - val_accuracy: 0.6574
Epoch 5/30
138/138 [==============================] - 119s 862ms/step - loss: 1.2181 - accuracy: 0.5792 - val_loss: 0.9978 - val_accuracy: 0.6556
Epoch 6/30
138/138 [==============================] - 117s 847ms/step - loss: 1.1457 - accuracy: 0.6083 - val_loss: 1.0130 - val_accuracy: 0.6704
